In [1]:
!pip install cssselect
!pip install pymongo
import re
import time
from typing import Iterator
import requests
import lxml.html
from pymongo import MongoClient

In [2]:
def generate_expansion(url):
    target1 = 'VER='
    target2 = '&CID='
    idx1 = url.find(target1)
    idx2 = url.find(target2)
    expansion = url[idx1 + 4: idx2]
    return expansion

def generate_key(url):
    target2 = '&CID='
    target3 = '&MODE'
    idx2 = url.find(target2)
    idx3 = url.find(target3)
    key = url[idx2 + 5: idx3]
    return key

def main():
    base_url = 'https://yuyu-tei.jp/game_poc/'
    session = requests.Session()
    res = session.get(base_url)
    html = lxml.html.fromstring(res.text)
    html.make_links_absolute(res.url)
    a_list = html.cssselect('#side > div > div.nav_box > ul:nth-child(2) > li > ul > li > ul > li > a')

    urls = []
    for a in a_list:
        url = a.get('href')
        urls.append(url)
        #yield urls

    for url in urls:
        session = requests.Session()
        time.sleep(1)
        res = session.get(url)
        html = lxml.html.fromstring(res.text)
        html.make_links_absolute(res.url)
        cards_url_element = html.cssselect('#main > div.card_list_box > div.group_box > ul > li > div.image_box > a') #各拡張ごとのカードのURL一覧が入った要素を取得

        cards_urls = [] #ここにURLを入れていく
        for url in cards_url_element:
            cards_url = url.get('href')    
            cards_urls.append(cards_url)
        

        for card_url in cards_urls:
            
            #すでにあるかを確認する
            key = generate_key(card_url)
            info = collection.find_one('key', key) 
            
            if not info: #Noneではなかった場合だけ詳細を見る。
                    res = session.get(card_url)
                    res.encoding = 'utf-8'
                    time.sleep(0.5)
                    html = lxml.html.fromstring(res.text)
            
                    price = int(
                        (html.cssselect('#main > div.card_detail_box > div.operation_box > div.price_box > form > p.price > b')
                        )[0].text.replace('円', ''))
                    rarity_name = html.cssselect('#main > div.card_detail_box > div.information_box > table.heading > tr:nth-child(2) > td')
                    rarity_name = rarity_name[0].text.strip().split(' ')
                    rarity = rarity_name[0]
                    name = rarity_name[1]

                    card_info = {
                        'url': card_url,
                        'key': generate_key(card_url),
                        'rarity': rarity,
                        'name': name,
                        'expansion': generate_expansion(card_url),
                        'price': price
                        }
                    collection.insert_one(card_info)
                    print(card_info)

In [3]:
client = MongoClient('localhost', 27017)
collection = client.scraping.card_info
collection.create_index('key', unique = True)

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 62ab1b5c5e008e40b9d33c9f, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>

In [14]:
main()

{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10146&MODE=sell', 'key': '10146', 'rarity': 'UR', 'name': 'ヒスイゾロアークVSTAR', 'expansion': 's10a', 'price': 4980, '_id': ObjectId('62ab1a9be476220317c7aac0')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10147&MODE=sell', 'key': '10147', 'rarity': 'UR', 'name': 'ダークパッチ', 'expansion': 's10a', 'price': 4980, '_id': ObjectId('62ab1a9ce476220317c7aac1')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10148&MODE=sell', 'key': '10148', 'rarity': 'UR', 'name': '災いの箱', 'expansion': 's10a', 'price': 1280, '_id': ObjectId('62ab1a9ce476220317c7aac2')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10139&MODE=sell', 'key': '10139', 'rarity': 'HR', 'name': 'ジバコイルVSTAR', 'expansion': 's10a', 'price': 780, '_id': ObjectId('62ab1a9de476220317c7aac3')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10140&

{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10053&MODE=sell', 'key': '10053', 'rarity': 'RR', 'name': 'ラブトロスV', 'expansion': 's10a', 'price': 50, '_id': ObjectId('62ab1ab8e476220317c7aae4')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10068&MODE=sell', 'key': '10068', 'rarity': 'RR', 'name': 'エルレイドV', 'expansion': 's10a', 'price': 50, '_id': ObjectId('62ab1ab9e476220317c7aae5')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10095&MODE=sell', 'key': '10095', 'rarity': 'RR', 'name': 'ヒスイヌメルゴンV', 'expansion': 's10a', 'price': 200, '_id': ObjectId('62ab1abae476220317c7aae6')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10102&MODE=sell', 'key': '10102', 'rarity': 'RR', 'name': 'ヒスイゾロアークV', 'expansion': 's10a', 'price': 680, '_id': ObjectId('62ab1abbe476220317c7aae7')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10010&MODE=sel

{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10084&MODE=sell', 'key': '10084', 'rarity': 'U', 'name': 'ドクロッグ(ミラー)', 'expansion': 's10a', 'price': 50, '_id': ObjectId('62ab1ad9e476220317c7ab09')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10088&MODE=sell', 'key': '10088', 'rarity': 'U', 'name': 'ドータクン', 'expansion': 's10a', 'price': 30, '_id': ObjectId('62ab1adae476220317c7ab0a')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10089&MODE=sell', 'key': '10089', 'rarity': 'U', 'name': 'ドータクン(ミラー)', 'expansion': 's10a', 'price': 50, '_id': ObjectId('62ab1adbe476220317c7ab0b')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10092&MODE=sell', 'key': '10092', 'rarity': 'U', 'name': 'ヒスイヌメイル', 'expansion': 's10a', 'price': 30, '_id': ObjectId('62ab1adbe476220317c7ab0c')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10093&MODE=sell', 

{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10024&MODE=sell', 'key': '10024', 'rarity': 'C', 'name': 'ピカチュウ', 'expansion': 's10a', 'price': 100, '_id': ObjectId('62ab1af9e476220317c7ab2e')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10025&MODE=sell', 'key': '10025', 'rarity': 'C', 'name': 'ピカチュウ(ミラー)', 'expansion': 's10a', 'price': 400, '_id': ObjectId('62ab1afae476220317c7ab2f')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10030&MODE=sell', 'key': '10030', 'rarity': 'C', 'name': 'ロトム', 'expansion': 's10a', 'price': 20, '_id': ObjectId('62ab1afbe476220317c7ab30')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10031&MODE=sell', 'key': '10031', 'rarity': 'C', 'name': 'ロトム(ミラー)', 'expansion': 's10a', 'price': 50, '_id': ObjectId('62ab1afbe476220317c7ab31')}
{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10032&MODE=sell', 'key

{'url': 'https://yuyu-tei.jp/game_poc/carddetail/cardpreview.php?VER=s10a&CID=10099&MODE=sell', 'key': '10099', 'rarity': 'C', 'name': 'エイパム(ミラー)', 'expansion': 's10a', 'price': 50, '_id': ObjectId('62ab1b18e476220317c7ab53')}


DuplicateKeyError: E11000 duplicate key error collection: scraping.card_info index: key_1 dup key: { key: "10146" }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'key': 1}, 'keyValue': {'key': '10146'}, 'errmsg': 'E11000 duplicate key error collection: scraping.card_info index: key_1 dup key: { key: "10146" }'}